In [ ]:
# draw function on 2d elements

import webgpu.jupyter
#from webgpu.mesh import MeshData
from ngsolve import *

def render(gpu, mesh, cf, order=2):
    import ngsolve_webgpu.mesh
    from webgpu import Draw
    import pyodide.ffi
    import ngsolve as ngs
    import js
    
    mesh_data = ngsolve_webgpu.mesh.MeshData(mesh, cf, order)
    mesh_object = ngsolve_webgpu.mesh.MeshRenderObject(gpu, mesh_data)
    point_numbers = ngsolve_webgpu.mesh.PointNumbersRenderObject(gpu, mesh_data, 13)
    gpu.colormap.set_min_max(0., 1.)
    Draw([mesh_object, point_numbers])

m = Mesh(unit_square.GenerateMesh(maxh=.1))
#m = Mesh(unit_cube.GenerateMesh(maxh=.3))
cf=sin(10*x)*sin(10*y)

# "data" will be passed as argument to render on the pyodide side
webgpu.jupyter.DrawCustom(render, kwargs={"cf": cf, "mesh": m, "order": 5}, modules=["ngsolve_webgpu"])

In [ ]:
# draw geometry

import webgpu.jupyter
from netgen.occ import *
from ngsolve import *

box = Box((-1,-1,-1), (1,1,1)) + Sphere((1,1,1), 0.4)

def setup(gpu, geo):
    import ngsolve_webgpu.geometry
    from webgpu.draw import Draw
    obj = ngsolve_webgpu.geometry.GeometryRenderObject(gpu, geo)
    Draw(obj)

def draw_geo(geo):
    if isinstance(geo, TopoDS_Shape):
        geo = OCCGeometry(geo)
    webgpu.jupyter.DrawCustom(setup, kwargs={"geo" : geo}, modules=["ngsolve_webgpu"])

draw_geo(box)

In [ ]:
# draw isosurface

import webgpu.jupyter
from netgen.occ import *
from ngsolve import *

box = Box((-1,-1,-1), (1,1,1))
geo = OCCGeometry(box)
mesh = Mesh(geo.GenerateMesh(maxh=.15))

cf = x**2 + y**2 + z**2 - 0.5**2

def setup(gpu, levelset, func, mesh):
    import ngsolve_webgpu.isosurface
    from webgpu import Draw
    Draw(ngsolve_webgpu.isosurface.IsoSurfaceRenderObject(gpu, levelset, func, mesh, "Isosurface"))

def draw_iso(levelset, func, mesh):
    webgpu.jupyter.DrawCustom(setup, kwargs={"levelset" : levelset, "mesh" : mesh, "func" : func}, modules=["ngsolve_webgpu"])

draw_iso(cf, x/Norm(CF((x,y,z))), mesh)